# Carbon Use Efficiency

Resources:
* Models: http://bigg.ucsd.edu/models/

In [1]:
import requests, cobra, tempfile, os, micom, pickle
from cue_tools import loadModelURL, computeCUE, CUEDef
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load

In [28]:
"""Download models locally"""
with open('spp.txt', 'r') as f:
    IDS = f.readlines()
    for ID in IDS:
        ID = ID.strip()
        url = f'http://bigg.ucsd.edu/static/models/{ID}.mat'
        with requests.get(url) as resp:
            with open(f'models/{ID}.mat', 'wb') as o:
                o.write(resp.content)

In [45]:
"""Setup MICOM"""
# Don't load these models due to metabolite naming bug
no = ['iJB785', 'iRC1080']
# Create input for MICOM
with open('spp.txt', 'r') as f:
    IDS = [x.strip() for x in f.readlines() if x.strip() not in no]
id_df = pd.DataFrame([{'id': ID, 'file': f'/Users/michaelsilverstein/Documents/Boston_University/Courses/bf571_cue/models/{ID}.mat'} for ID in IDS])

# Compute CUE for mono and co-cultures
Assumptions:
* Even communities (50% abundance for both organisms)
* Non-fixed media: medias are model dependent and co-culture medias are a combination of medias from individual models


In [47]:
%%time
"""Compute CUE for mono and co-cultures"""
# Keep track of when mono_cue has been calculated
mono_ids = []
# Save community CUEs
N = len(IDS)
com_data = []
for i in range(N):
    id1 = IDS[i]
    for j in range(N):
        if i == j:
            continue
            
        id2 = IDS[j]
        # Create community
        p_df = id_df[id_df.id.isin([id1, id2])]
        community = micom.Community(p_df)
        
        # Add monoculture CUE calculations
        missing = {id1, id2} - set(mono_ids)
        for ID in missing:
            # Ideally, community.optimize_single(ID, fluxes=True) would work...
            m = cobra.io.load_matlab_model(f'models/{ID}.mat')
            mcue = CUEDef(m).squeeze()['Def 4']
            # Save
            data = {'ids': {ID}, 'cue': mcue}
            com_data.append(data)
            mono_ids.append(ID)
            
        # Compute community cue
        sol = community.optimize(fluxes=True)
        co2 = sol.fluxes.loc['medium', 'EX_co2_m']
        biomass = sol.objective_value
        com_cue = biomass / (biomass + co2)
        
        # Save
        data = {'ids': {id1, id2}, 'cue': com_cue}
        com_data.append(data)

# Convert to df
cue_df = pd.DataFrame(com_data)
cue_df.head()

KeyboardInterrupt: 

In [219]:
"""Load total community"""
community = micom.Community(id_df)
# community._rtol = 0

In [221]:
media = micom.media.minimal_medium(community, 1)
community.medium = media

In [246]:
id1 = IDS[5]
id2 = IDS[5]

# Set abundance to subset
# abunds = pd.Series({ID: 1 if ID in {id1, id2} else 0 for ID in community.taxa})
abunds = pd.Series({ID: 1 for ID in community.taxa})
community.set_abundance(abunds)

sol = community.optimize(fluxes=True)


In [195]:
[i for i in sol.fluxes if 'co2'in i]

['EX_co2_e', 'EX_co2_m']

In [223]:
sol.fluxes.loc['medium', 'EX_co2_m']

9.817779118742187e-05

In [236]:
sol.fluxes.loc['medium', [i for i in sol.fluxes if 'co2' in i.lower()]]

reaction
CO2t             NaN
CO2tex           NaN
CO2tg            NaN
CO2tm            NaN
CO2tn            NaN
CO2tp            NaN
CO2tpp           NaN
CO2tv            NaN
EX_co2_e         NaN
EX_co2_m    5.607498
Name: medium, dtype: float64

In [244]:
micom.media.minimal_medium(community, 1, minimize_components=True )

KeyboardInterrupt: 

In [243]:
cobra.medium.minimal_medium()

<module 'cobra.medium' from '/Users/michaelsilverstein/anaconda3/lib/python3.8/site-packages/cobra/medium/__init__.py'>

In [242]:
t = cobra.io.load_matlab_model(f'models/{id1}.mat')

In [224]:
t.medium = media

KeyError: 'EX_pi_m'

In [216]:
t.optimize().fluxes.EX_co2_e

12.606061434738258

In [174]:
community._rtol = 0

In [245]:
community.optimize()